In [1]:
import torch
import circuits as ckt
from models import Solver
from models import State
from data import Input,Process

In [2]:
# Get cpu or gpu device for training.
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
print(f"Using {device} device")

# torch.autograd.set_detect_anomaly(True)
torch.set_printoptions(precision=2, linewidth=200)

Using cpu device


# Generate Input Data for a Simple Circuit
Circuit is an independent voltage source and a resistor.

In [3]:
vr = ckt.Circuit()

v_source = vr.add_element(kind=ckt.Kinds.IVS)
r1 = vr.add_element(kind=ckt.Kinds.R)
v_source.connect(v_source.high, r1.high)
v_source.connect(v_source.low, r1.low)
# Case 1 all resistances known
# v_source.attr = 1
# r1.attr = 1e6
# Case 2 one resisance unknown
v_source.attr = 1
v_source.i = -10
# Case 3 one voltage and one current known
# v_source.attr = 1
# r1.i = 10

# v_source = vr.add_element(kind=ckt.Kinds.IVS)
# r1 = vr.add_element(kind=ckt.Kinds.R)
# r2 = vr.add_element(kind=ckt.Kinds.R)
# v_source.connect(v_source.high, r1.high)
# r1.connect(r1.low, r2.high)
# v_source.connect(v_source.low, r2.low)
# # # Case 1 all resistances known
# v_source.attr = 1
# r1.attr = 1e6
# r2.attr = 1e6
# # # Case 2 one resisance unknown
# # v_source.attr = 1
# # r1.attr = 10
# # r2.v = 0.5

In [4]:
input = Input(vr)

c:\Users\terry\OneDrive\Documents\GitHub\side_circuit\ml\circuits.py:65: FutureWarning: incidence_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  M_scipy = nx.incidence_matrix(G=self.nx_graph(),oriented=True)


In [5]:
model = Solver(input=input,attr=input.attr_param, state=State.Solve)
X = model()
X

tensor([[-44.29],
        [ 44.29],
        [  1.00],
        [  1.00],
        [ -1.00]], grad_fn=<LinalgSolveExBackward0>)

In [6]:
input.ivp_inputs()

tensor([[-10.],
        [  0.],
        [  1.],
        [  0.],
        [  0.],
        [  0.]])

In [7]:
input.ivp_knowns_mask()

tensor([[ True],
        [False],
        [ True],
        [False],
        [False],
        [False]])

In [8]:
process = Process(input)

In [9]:
process.errors(X)

tensor([[ 34.29],
        [-44.29],
        [  0.00],
        [ -1.00],
        [  1.00]], grad_fn=<SubBackward0>)